**Imports**

In [ ]:
import json
from pprint import pprint
import pandas as pd
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import pickle

from pathlib import Path

from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

**NLTK Downloads**

In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')

# Constructing the Data Set

In [ ]:
data_folder = Path('data/twint')
tweets_file = data_folder / 'data_new2.txt'

In [ ]:
f = open(tweets_file, 'r')
lines = [l for l in f]
f.close()

In [ ]:
tweets = [json.loads(l) for l in lines]
print('Number of tweets read in:\t%s' % len(tweets))

# Community Detection

## Data Filtering and Pre-Processing

In [ ]:
stop_words = stopwords.words('english')
tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [ ]:
def word_tokenize(tweet):
    return tokenizer.tokenize(tweet)

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# https://stackoverflow.com/a/49146722/330558
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    #Remove numbers
    tweet = re.sub(r"[0-9]", '', tweet, flags=re.MULTILINE)
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove punctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove emojis
    tweet = remove_emoji(tweet)
    # Tokenize
    tweet_tokens = word_tokenize(tweet)
    # Remove stopwords
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    # Stemming
    #shortened = [stemmer.stem(w) for w in filtered_words]
    # Lemmatizing
    lemmatized = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in filtered_words]
    # Remove non-alpha words
    lemmatized_filtered = [w for w in lemmatized if w.isalpha()]
    # Filter out short lemmas
    final_tokens = [w for w in lemmatized_filtered if len(w) > 2]
    
    return final_tokens

In [ ]:
unique_usernames = {t['username'] for t in tweets}
users_to_documents = {u:[] for u in unique_usernames}

for tw in tweets:
    u = tw['username']
    t = preprocess_tweet_text(tw['tweet'])
    users_to_documents[u] = users_to_documents[u] + t

In [ ]:
print(len(users_to_documents.keys()))

## Document Length Distribution

In [ ]:
lengths = {u:len(doc) for (u, doc) in users_to_documents.items()}
min = np.min(list(lengths.values()))
max = np.max(list(lengths.values()))
bins = np.linspace(start=min, stop=max, num=300, endpoint=True)

labels = list(range(299))

lengths_df = pd.DataFrame({'Doc': list(lengths.keys()), 'Document Length': list(lengths.values())})

ax = lengths_df.hist(figsize=(20, 4), grid=False, bins=len(bins), range=(30, 200), rwidth=0.8)
ax[0][0].set_title('Document Length Distribution', fontsize=20)
ax[0][0].set_xlabel('Document Length', fontsize=20)
ax[0][0].set_ylabel('Number of Documents', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.savefig('document_length_distribution.pdf', bbox_inches='tight')
plt.show()
plt.close()

### Filter out any users that now have less than 120 tokens

In [ ]:
print(len(users_to_documents.values()))
users_to_documents_doc_length_filtered = {u:doc for (u, doc) in users_to_documents.items() if len(doc) >= 120}
print(len(users_to_documents_doc_length_filtered.values()))

## Lexical diversity

In [ ]:
def lexical_diversity_percentage(tokens):
    return len(set(tokens)) / len(tokens) * 100

In [ ]:
lexical_diversities = {u:lexical_diversity_percentage(doc) for (u, doc) in users_to_documents_doc_length_filtered.items()}
lexical_div_df = pd.DataFrame({'Doc': list(lexical_diversities.keys()), 'Lexical Diversity': list(lexical_diversities.values())})
ax = lexical_div_df.hist(grid=False, bins=50, figsize=(20, 10), rwidth=0.8)
ax[0][0].set_title('Lexical Diversity Distribution', fontsize=20)
ax[0][0].set_xlabel('Lexical Diversity', fontsize=20)
ax[0][0].set_ylabel('Number of Documents', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.savefig('lexical_diversity_distribution.pdf', bbox_inches='tight')
plt.show()
plt.close()

### Filter out any users with a very low lexical diversity

In [ ]:
print(len(users_to_documents_doc_length_filtered))
users_to_documents_filtered = {u:doc for (u, doc) in users_to_documents_doc_length_filtered.items() if lexical_diversity_percentage(doc) >= 48}
len(users_to_documents_filtered)

## Construction of the Tf-Idf Matrix

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
docs = users_to_documents_filtered.values()

In [ ]:
def do_nothing(tokens):
    return tokens

vectorizer = TfidfVectorizer(lowercase=False, tokenizer=do_nothing, min_df=0.2, max_df=0.8)

X = vectorizer.fit_transform(docs)

In [ ]:
tf_idf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
tf_idf_df

In [ ]:
tf_idf_df.iloc[:10, 60:70]

In [ ]:
tfidf_file = open('tfidf.pickle', 'wb')
pickle.dump(tf_idf_df, tfidf_file)
tfidf_file.close()

## Optional: Load the tf-idf Matrix from Pickle

In [ ]:
tfidf_file = open('tfidf.pickle', 'rb')
tf_idf_df = pickle.load(tfidf_file)
tfidf_file.close()

# Perform the PCA

In [ ]:
from sklearn import preprocessing
data_scaled = pd.DataFrame(preprocessing.normalize(tf_idf_df), columns=tf_idf_df.columns)

In [ ]:
pca = PCA(n_components=30)
X_reduced = pca.fit_transform(data_scaled)

In [ ]:
len(X_reduced)

## Analyze the Percentage of explained Variance per Principal Component

In [ ]:
pd.DataFrame(data={'PCA': [i+1 for i in range(0, len(pca.explained_variance_ratio_))],'Explained Variance': pca.explained_variance_ratio_})

In [ ]:
explained_variance = pca.explained_variance_ratio_.sum()
print("Explained variance of the PCA step: {}%".format(int(explained_variance * 100)))

In [ ]:
plt.figure(figsize=(20, 10))
ax = plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.title('Explained Variance in Relation to the Number of Principal Components', fontsize=20)
plt.xlabel('Explained Variance in Percentage', fontsize=20)
plt.ylabel('Number of Principal Components', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.savefig('explained_variance.pdf', bbox_inches='tight')
plt.show()
plt.close()

## 2D Plot

In [ ]:
plt.figure(1, figsize=(14, 14))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], color='lightblue', edgecolor='black', s=40)
plt.title('Reduced Data Set', fontsize=20)
plt.xlabel('PC 1', fontsize=20)
plt.ylabel('PC 2', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.savefig('pca_first_2_pcs.pdf', bbox_inches='tight')
plt.show()
plt.close()

## 3D Plot

In [ ]:
fig = plt.figure(1, figsize=(14, 14))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = Axes3D(fig, elev=-150, azim=110)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], color='lightblue', edgecolor='black', s=40)
ax.set_title('Reduced Data Set', fontsize=20)
ax.set_xlabel('PC 1', fontsize=20, labelpad=40)
ax.set_ylabel('PC 2', fontsize=20, labelpad=40)
ax.set_zlabel('PC 3', fontsize=20, labelpad=40)
ax.tick_params(labelsize=20, pad=20)
plt.savefig('pca_first_3_pcs.pdf', bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important_feature_indexes_per_pc = {}

initial_feature_names = data_scaled.columns

dfs = []

for pc in range(n_pcs):
    components_enumerated = list(enumerate(pca.components_[pc]))
    components_sorted = sorted(components_enumerated, key=lambda x:np.abs(x[1]), reverse=True)
    features_imps = {initial_feature_names[i]:imp for (i, imp) in components_sorted}
    features_imps_df = pd.DataFrame.from_dict(features_imps, orient='index', columns = ['importance'] )[:10]
    dfs.append(features_imps_df)
    
print(dfs[0])
print()
print(dfs[1])
print()
print(dfs[2])

# K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

## Perfoming the Clustering

In [ ]:
km = KMeans(n_clusters=3)
km.fit(X_reduced)

## 2D Plot

In [ ]:
y_kmeans = km.labels_
plt.figure(figsize=(14, 14))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y_kmeans, s=40, edgecolor='black')
centers = km.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=400, alpha=0.8, edgecolor='black')
plt.tick_params(labelsize=20)
plt.xlabel('PC 1', fontsize=20)
plt.ylabel('PC 2', fontsize=20)
plt.title('K-Means Result', fontsize=20)
plt.savefig('k_means_result_2d.pdf', bbox_inches='tight')
plt.show()
plt.close()

## 3D Plot

In [ ]:
fig = plt.figure(1, figsize=(14, 14))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = Axes3D(fig, elev=-150, azim=110)
y = range(0, len(X_reduced))
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y_kmeans, edgecolor='k', s=50)
ax.scatter(centers[:, 0], centers[:, 1], centers[:, 2], c='red', s=400, alpha=0.8, edgecolor='black')
ax.set_title('K-Means Result', fontsize=20)

ax.set_xlabel('PC 1', fontsize=20, labelpad=40)
ax.set_ylabel('PC 2', fontsize=20, labelpad=40)
ax.set_zlabel('PC 3', fontsize=20, labelpad=40)
ax.tick_params(labelsize=20, pad=20)
plt.savefig('k_means_result_3d.pdf', bbox_inches='tight')
plt.show()
plt.close()

## Get the Users per Clusters

In [ ]:
users_df = pd.DataFrame({'Username': list(users_to_documents_filtered.keys()), 'Cluster Label': km.labels_})

for i in range(0, len(users_df)):
    p = X_reduced[i]
    cluster = users_df.loc[i]['Cluster Label']
    
users_cluster_0 = users_df[users_df['Cluster Label'] == 0]
users_cluster_1 = users_df[users_df['Cluster Label'] == 1]
users_cluster_2 = users_df[users_df['Cluster Label'] == 2]

## Retweets per individual user for each cluster

In [ ]:
retweet_count_per_users_cluster_0 = {u:0 for u in list(users_cluster_0['Username'])}
retweet_count_per_users_cluster_1 = {u:0 for u in list(users_cluster_1['Username'])}
retweet_count_per_users_cluster_2 = {u:0 for u in list(users_cluster_2['Username'])}

for t in tweets:
    username = t['username']
    retweets = t['retweets_count']
    if username in retweet_count_per_users_cluster_0:
        retweet_count_per_users_cluster_0[username] += retweets
    if username in retweet_count_per_users_cluster_1:
        retweet_count_per_users_cluster_1[username] += retweets
    if username in retweet_count_per_users_cluster_2:
        retweet_count_per_users_cluster_2[username] += retweets
        
ten_most_retweeted_cluster_0 = pd.DataFrame({'Username': list(retweet_count_per_users_cluster_0.keys()), 'Retweets': list(retweet_count_per_users_cluster_0.values())}).sort_values(by='Retweets')[::-1].head(15)
ten_most_retweeted_cluster_1 = pd.DataFrame({'Username': list(retweet_count_per_users_cluster_1.keys()), 'Retweets': list(retweet_count_per_users_cluster_1.values())}).sort_values(by='Retweets')[::-1].head(15)
ten_most_retweeted_cluster_2 = pd.DataFrame({'Username': list(retweet_count_per_users_cluster_2.keys()), 'Retweets': list(retweet_count_per_users_cluster_2.values())}).sort_values(by='Retweets')[::-1].head(15)


tweets_cluster_0 = []
tweets_cluster_1 = []
tweets_cluster_2 = []

for t in tweets:
    username = t['username']
    tweet = t['tweet']
    if username in list(ten_most_retweeted_cluster_0['Username']):
        tweets_cluster_0.append(t)
    if username in list(ten_most_retweeted_cluster_1['Username']):
        tweets_cluster_1.append(t)
    if username in list(ten_most_retweeted_cluster_2['Username']):
        tweets_cluster_2.append(t)

In [ ]:
print(ten_most_retweeted_cluster_0)
print()
print(ten_most_retweeted_cluster_1)
print()
print(ten_most_retweeted_cluster_2)

## VADER Sentiment Analysis

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
sentiments_cluster_0_df = pd.DataFrame(columns=['Link', 'Sentence', 'Pos', 'Neu', 'Neg', 'Compound'])
sentiments_cluster_1_df = pd.DataFrame(columns=['Link', 'Sentence', 'Pos', 'Neu', 'Neg', 'Compound'])
sentiments_cluster_2_df = pd.DataFrame(columns=['Link', 'Sentence', 'Pos', 'Neu', 'Neg', 'Compound'])


for t in tweets_cluster_0:
    ss = sid.polarity_scores(t['tweet'])
    sentiments_cluster_0_df = sentiments_cluster_0_df.append({'Link': t['link'],
                                                              'Sentence': t['tweet'],
                                                             'Pos': ss['pos'],
                                                             'Neu': ss['neu'],
                                                             'Neg': ss['neg'],
                                                             'Compound': ss['compound']}, ignore_index=True)
    
for t in tweets_cluster_1:
    ss = sid.polarity_scores(t['tweet'])
    sentiments_cluster_1_df = sentiments_cluster_1_df.append({'Link': t['link'],
                                                              'Sentence': t['tweet'],
                                                             'Pos': ss['pos'],
                                                             'Neu': ss['neu'],
                                                             'Neg': ss['neg'],
                                                             'Compound': ss['compound']}, ignore_index=True)

for t in tweets_cluster_2:
    ss = sid.polarity_scores(t['tweet'])
    sentiments_cluster_2_df = sentiments_cluster_2_df.append({'Link': t['link'],
                                                              'Sentence': t['tweet'],
                                                             'Pos': ss['pos'],
                                                             'Neu': ss['neu'],
                                                             'Neg': ss['neg'],
                                                             'Compound': ss['compound']}, ignore_index=True)

In [ ]:
sentiments_cluster_2_df.sort_values(by='Compound')[::-1].head(10).values

## Get the tweets per cluster

In [ ]:
tweets_cluster0 = [t for t in tweets if t['username'] in list(users_cluster_0['Username'])]    
tweets_cluster1 = [t for t in tweets if t['username'] in list(users_cluster_1['Username'])]
tweets_cluster2 = [t for t in tweets if t['username'] in list(users_cluster_2['Username'])]
texts_cluster0 = [t['tweet'] for t in tweets_cluster0]
texts_cluster1 = [t['tweet'] for t in tweets_cluster1]
texts_cluster2 = [t['tweet'] for t in tweets_cluster2]
print(len(texts_cluster0))
print(len(texts_cluster1))
print(len(texts_cluster2))

## Most used hashtags in the cluster

In [ ]:
from collections import Counter

hashtags_cluster0 = [h for t in tweets_cluster0 for h in t['hashtags']]
hashtags_cluster1 = [h for t in tweets_cluster1 for h in t['hashtags']]
hashtags_cluster2 = [h for t in tweets_cluster2 for h in t['hashtags']]


c0 = Counter(hashtags_cluster0)
c1 = Counter(hashtags_cluster1)
c2 = Counter(hashtags_cluster2)

df0 = pd.DataFrame(index=[e[0] for e in c0.most_common(10)], data={'count': [e[1] for e in c0.most_common(10)]}).sort_values(by='count')[::-1]
df1 = pd.DataFrame(index=[e[0] for e in c1.most_common(10)], data={'count': [e[1] for e in c1.most_common(10)]}).sort_values(by='count')[::-1]
df2 = pd.DataFrame(index=[e[0] for e in c2.most_common(10)], data={'count': [e[1] for e in c2.most_common(10)]}).sort_values(by='count')[::-1]

fig, axes = plt.subplots(3, 1, figsize=(22, 13))

plt.subplots_adjust(hspace=2)

df0.plot(ax=axes[0], kind='bar', title='Most used Hashtags in Cluster 1', legend=False, fontsize=20)
df1.plot(ax=axes[1], kind='bar', title='Most used Hashtags in Cluster 2', legend=False, fontsize=20)
df2.plot(ax=axes[2], kind='bar', title='Most used Hashtags in Cluster 3', legend=False, fontsize=20)

for i in range(3):
    axes[i].title.set_size(20)
    axes[i].tick_params(axis='x', labelrotation=45)

plt.savefig('most_used_hashtags_per_cluster.png', bbox_inches='tight')
plt.show()
plt.close()

## Number of Tweets per day per cluster

In [ ]:
clusters = [tweets_cluster0, tweets_cluster1, tweets_cluster2]

import matplotlib.pyplot as plt
fig, axes = plt.subplots(3,1, figsize=(20, 13))
plt.subplots_adjust(hspace=0.6)
i = 0
for c in clusters:
    dates_to_n_tweets = {}
    for t in c:
        date = t['date'].split('-')[1] + '-' + t['date'].split('-')[2]
        try:
            dates_to_n_tweets[date] = dates_to_n_tweets[date] + 1
        except:
            dates_to_n_tweets[date] = 0

    sorted_dates = sorted(dates_to_n_tweets.items())
    df = pd.DataFrame({'date': [e[0] for e in sorted_dates], 'count': [e[1] for e in sorted_dates]})
    ax = df.plot(ax=axes[i], kind='line', title='Number of Tweets per Day in Cluster %d' % (i+1), legend=False, fontsize=20)
    xlabels = df['date']
    ax.set_xticklabels(xlabels, fontsize=14, rotation=90)
    ax.set_xticks(range(len(xlabels)))
    ax.axvline(x=37, color='red', alpha=0.5)
    ax.axvline(x=39, color='red', alpha=0.5)
    i+=1
    
axes[0].title.set_size(20)
axes[1].title.set_size(20)
axes[2].title.set_size(20)

plt.savefig('tweets_per_day_per_cluster.png', bbox_inches='tight')
plt.show()
plt.close()

## Four most liked tweets per cluster

In [ ]:
tweets_cluster0_sorted = sorted(tweets_cluster0, key=lambda t: t['likes_count'], reverse=True)
tweets_cluster1_sorted = sorted(tweets_cluster1, key=lambda t: t['retweets_count'], reverse=True)
tweets_cluster2_sorted = sorted(tweets_cluster2, key=lambda t: t['retweets_count'], reverse=True)

#print(*[t for t in tweets_cluster0_sorted[:4]], sep='\n_________________________________________\n\n\n')
#print('::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::')
#print(*[t for t in tweets_cluster1_sorted[:4]], sep='\n_________________________________________\n\n\n')
#print('::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::')
print(*[t for t in tweets_cluster2_sorted[:4]], sep='\n_________________________________________\n\n\n')

# Shifterator Analysis

In [ ]:
import shifterator as sh

In [ ]:
tokens_cluster0 = [t for (u, doc) in users_to_documents_doc_length_filtered.items() if u in list(users_cluster_0['Username']) for t in doc]    
tokens_cluster1 = [t for (u, doc) in users_to_documents_doc_length_filtered.items() if u in list(users_cluster_1['Username']) for t in doc]    
tokens_cluster2 = [t for (u, doc) in users_to_documents_doc_length_filtered.items() if u in list(users_cluster_2['Username']) for t in doc]    

In [ ]:
token_freqs_cluster_0 = dict(Counter(tokens_cluster0))
token_freqs_cluster_1 = dict(Counter(tokens_cluster1))
token_freqs_cluster_2 = dict(Counter(tokens_cluster2))

In [ ]:
shift = sh.EntropyShift(type2freq_1=token_freqs_cluster_0, type2freq_2=token_freqs_cluster_1)
shift.get_shift_graph(system_names = ['Cluster 1', 'Cluster 2'], filename='entropy_shift_cluster1_cluster2.png')